# Pentathlon-III: Next Product to Buy Models

* Team-lead GitLab userid: yil012
* Group name: Anaconda
* Team member names: Yinluo Li, Yuquan Zheng, Zezheng Hao, Shangfu Chen

In [1]:
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit
from utils import functions

warnings.filterwarnings("ignore")

In [2]:
## loading the data - this dataset must NOT be changed
pentathlon_nptb = pd.read_pickle("data/pentathlon_nptb.pkl")
pentathlon_nptb["buyer_yes"] = (pentathlon_nptb["buyer"] == "yes").astype(int)
pentathlon_nptb.head()

,custid,buyer,total_os,message,age,gender,income,education,children,freq_endurance,...,endurance_os,strength_os,water_os,team_os,backcountry_os,winter_os,racquet_os,training,representative,buyer_yes
0,U45198803,no,0.0,endurance,30 to 44,M,25000,14,1.3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
1,U22197752,no,0.0,backcountry,45 to 59,F,40000,44,0.4,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
2,U83874832,no,0.0,backcountry,45 to 59,M,50000,24,0.8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
3,U19423462,no,0.0,winter,45 to 59,F,50000,26,1.1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
4,U23888305,no,0.0,winter,30 to 44,M,40000,22,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0


In [3]:
pentathlon_nptb_copy = pentathlon_nptb.copy()

In [4]:
# preprocessing data
pentathlon_nptb_copy = pentathlon_nptb_copy.assign(
    income_ln=np.log(pentathlon_nptb_copy["income"] + 1),
    cweight=rsm.ifelse(pentathlon_nptb.buyer == "yes", 1, 99),
)
pentathlon_nptb_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   custid            200000 non-null  object  
 1   buyer             200000 non-null  category
 2   total_os          200000 non-null  float64 
 3   message           200000 non-null  category
 4   age               200000 non-null  category
 5   gender            200000 non-null  category
 6   income            200000 non-null  int32   
 7   education         200000 non-null  int32   
 8   children          200000 non-null  float64 
 9   freq_endurance    200000 non-null  int32   
 10  freq_strength     200000 non-null  int32   
 11  freq_water        200000 non-null  int32   
 12  freq_team         200000 non-null  int32   
 13  freq_backcountry  200000 non-null  int32   
 14  freq_winter       200000 non-null  int32   
 15  freq_racquet      200000 non-null  int32   
 16  en

In [5]:
# split data set
pentathlon_nptb_train = pentathlon_nptb_copy.query(
    "training == 1 & representative == 0"
)
pentathlon_nptb_test = pentathlon_nptb_copy.query("training == 0 & representative == 0")

# Logistic Regression

In [6]:
lr = smf.glm(
    formula="buyer_yes ~ message + age + gender + income_ln + \
             freq_endurance + freq_strength + freq_water + freq_team + freq_backcountry + freq_winter + freq_racquet + \
             message:age + message:gender + message:income_ln + message:freq_endurance + \
             message:freq_strength + message:freq_water + message:freq_team + message:freq_backcountry + \
             message:freq_winter + message:freq_racquet",
    family=Binomial(link=logit()),
    data=pentathlon_nptb_train,
    freq_weights=pentathlon_nptb_train["cweight"],
).fit(cov_type="HC1")
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              buyer_yes   No. Observations:                70000
Model:                            GLM   Df Residuals:                  3499909
Model Family:                Binomial   Df Model:                           90
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5160e+05
Date:                Thu, 25 Feb 2021   Deviance:                   3.0320e+05
Time:                        08:06:39   Pearson chi2:                 4.72e+06
No. Iterations:                     9                                         
Covariance Type:                  HC1                                         
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -31.5320      0.766    -41.159      0.000     -33.033     -30.030
message[T.strength]                        -2.8937      1.050     -2.757      0.006      -4.951      -0.837
message[T.water]                            4.1038      1.059      3.876      0.000       2.028       6.179
message[T.team]                             2.5505      1.046      2.438      0.015       0.500       4.601
message[T.backcountry]                     -0.7903      1.092     -0.724      0.469      -2.930       1.349
message[T.winter]                          -0.2803      1.071     -0.262      0.794      -2.379       1.818
message[T.racquet]                          1.7125      1.081      1.584      0.113      -0.407       3.832
age[T.30 to 44]                             0.6471      0.051     12.566      0.000       0.546       0.748
age[T.45 to 59]                             0.7770      0.053     14.715      0.000       0.673       0.880
age[T.>= 60]                                0.1035      0.066      1.564      0.118      -0.026       0.233
gender[T.F]                                -0.3754      0.034    -10.959      0.000      -0.442      -0.308
message[T.strength]:age[T.30 to 44]         0.3640      0.074      4.913      0.000       0.219       0.509
message[T.water]:age[T.30 to 44]            0.0730      0.074      0.989      0.323      -0.072       0.218
message[T.team]:age[T.30 to 44]            -0.0251      0.075     -0.334      0.739      -0.172       0.122
message[T.backcountry]:age[T.30 to 44]      0.0129      0.077      0.169      0.866      -0.137       0.163
message[T.winter]:age[T.30 to 44]           0.3467      0.076      4.589      0.000       0.199       0.495
message[T.racquet]:age[T.30 to 44]          0.0489      0.076      0.647      0.518      -0.099       0.197
message[T.strength]:age[T.45 to 59]        -0.1492      0.076     -1.969      0.049      -0.298      -0.001
message[T.water]:age[T.45 to 59]           -0.1120      0.077     -1.454      0.146      -0.263       0.039
message[T.team]:age[T.45 to 59]            -0.0554      0.077     -0.719      0.472      -0.206       0.096
message[T.backcountry]:age[T.45 to 59]      0.0746      0.078      0.957      0.339      -0.078       0.227
message[T.winter]:age[T.45 to 59]           0.0095      0.077      0.123      0.902      -0.142       0.161
message[T.racquet]:age[T.45 to 59]         -0.1486      0.078     -1.897      0.058      -0.302       0.005
message[T.strength]:age[T.>= 60]           -0.0763      0.094     -0.813      0.416      -0.260       0.108
message[T.water]:age[T.>= 60]               0.1465      0.093      1.573      0.116      -0.036       0.329
message[T.team]:age[T.>= 60]                0.0017      0.0

## AUC on test set

In [7]:
# make prediction by using current message option
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_yes_lr=lr.predict(pentathlon_nptb_test)
)
## calculate auc on testing set
print(f"AUC on test set is {functions.cal_auc(pentathlon_nptb_test, 'p_yes_lr')}")

AUC on test set is 0.876


## Prediction on test set

In [8]:
# 'freq_endurance', 'freq_strength', 'freq_water', 'freq_team', 'freq_backcountry', 'freq_winter', 'freq_racquet',
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_endurance_lr=lr.predict(pentathlon_nptb_test.assign(message="endurance")),
    p_strength_lr=lr.predict(pentathlon_nptb_test.assign(message="strength")),
    p_water_lr=lr.predict(pentathlon_nptb_test.assign(message="water")),
    p_team_lr=lr.predict(pentathlon_nptb_test.assign(message="team")),
    p_backcountry_lr=lr.predict(pentathlon_nptb_test.assign(message="backcountry")),
    p_winter_lr=lr.predict(pentathlon_nptb_test.assign(message="winter")),
    p_racquet_lr=lr.predict(pentathlon_nptb_test.assign(message="racquet")),
)

In [9]:
p_set_lr = [
    "p_endurance_lr",
    "p_strength_lr",
    "p_water_lr",
    "p_team_lr",
    "p_backcountry_lr",
    "p_winter_lr",
    "p_racquet_lr",
]
pentathlon_nptb_test["p_target_lr"] = pentathlon_nptb_test[p_set_lr].max(axis=1)

In [10]:
# # choose the message that has the highest probability
pentathlon_nptb_test["to_offer_lr"] = (
    pentathlon_nptb_test[p_set_lr]
    .idxmax(axis=1)
    .str.replace("p_endurance_lr", "endurance")
    .replace("p_strength_lr", "strength")
    .replace("p_water_lr", "water")
    .replace("p_team_lr", "team")
    .replace("p_backcountry_lr", "backcountry")
    .replace("p_winter_lr", "racquet")
    .replace("p_racquet_lr", "winter")
)

In [11]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_lr"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_lr,
backcountry,262
endurance,"9,310"
racquet,"1,756"
strength,"7,401"
team,"3,135"
water,"7,989"
winter,147


In [12]:
pentathlon_nptb_test["p_target_lr"] = pentathlon_nptb_test[p_set_lr].max(axis=1)
pentathlon_nptb_test[
    [
        "p_endurance_lr",
        "p_strength_lr",
        "p_water_lr",
        "p_team_lr",
        "p_backcountry_lr",
        "p_winter_lr",
        "p_racquet_lr",
        "p_target_lr",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, perc=True)

p_target_lr         6.98%
p_endurance_lr      5.88%
p_strength_lr       5.86%
p_winter_lr         5.33%
p_racquet_lr         4.9%
p_water_lr          4.64%
p_backcountry_lr    4.63%
p_team_lr           4.55%
dtype: object

## Profit on test set

In [13]:
ep_set_lr = [
    "ep_endurance_lr",
    "ep_strength_lr",
    "ep_water_lr",
    "ep_team_lr",
    "ep_backcountry_lr",
    "ep_winter_lr",
    "ep_racquet_lr",
]

In [14]:
avg_os = (
    pentathlon_nptb_copy.query("training == 1 & representative == 0 & buyer == 'yes'")
    .groupby("message")
    .total_os.mean()
    * 0.4
)

In [15]:
for i in range(7):
    pentathlon_nptb_test[ep_set_lr[i]] = pentathlon_nptb_test[p_set_lr[i]] * avg_os[i]
pentathlon_nptb_test["to_offer_ep_lr"] = (
    pentathlon_nptb_test[ep_set_lr]
    .idxmax(axis=1)
    .str.replace("ep_endurance_lr", "endurance")
    .replace("ep_strength_lr", "strength")
    .replace("ep_water_lr", "water")
    .replace("ep_team_lr", "team")
    .replace("ep_backcountry_lr", "backcountry")
    .replace("ep_winter_lr", "racquet")
    .replace("ep_racquet_lr", "winter")
)
pentathlon_nptb_test["ep_target_lr"] = pentathlon_nptb_test[ep_set_lr].max(axis=1)

In [16]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_ep_lr"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep_lr,
backcountry,"1,308"
endurance,"6,828"
racquet,"1,974"
strength,"7,011"
team,"3,577"
water,"9,026"
winter,276


In [17]:
pentathlon_nptb_test[
    [
        "ep_endurance_lr",
        "ep_strength_lr",
        "ep_water_lr",
        "ep_team_lr",
        "ep_backcountry_lr",
        "ep_winter_lr",
        "ep_racquet_lr",
        "ep_target_lr",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target_lr         $1.57
ep_strength_lr       $1.31
ep_endurance_lr      $1.28
ep_winter_lr         $1.26
ep_racquet_lr        $1.13
ep_backcountry_lr    $1.12
ep_water_lr          $1.06
ep_team_lr           $1.05
dtype: object

In [18]:
profit_lr = pentathlon_nptb_test["ep_target_lr"].agg("mean") * 30000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_lr)}")

Expected profit from offer customization: $47,179.17


# Random Forest

In [19]:
variables_rf = [
    "message",
    "age",
    "gender",
    "income_ln",
    "education",
    "children",
    "freq_endurance",
    "freq_strength",
    "freq_water",
    "freq_team",
    "freq_backcountry",
    "freq_winter",
    "freq_racquet",
]

X_train_rf = pentathlon_nptb_train[variables_rf]
X_train_rf = pd.get_dummies(X_train_rf, drop_first=True)
y_train_rf = pentathlon_nptb_train["buyer_yes"]

X_test_rf = pentathlon_nptb_test[variables_rf]
X_test_rf = pd.get_dummies(X_test_rf, drop_first=True)
y_test_rf = pentathlon_nptb_test["buyer_yes"]

In [122]:
rf = RandomForestClassifier(
    bootstrap=True, random_state=1234, criterion="entropy", max_depth=10
).fit(
    X_train_rf,
    y_train_rf,
    sample_weight=pentathlon_nptb_train["cweight"],
)

## AUC on test set

In [123]:
# make prediction by using current message option
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_yes_rf=rf.predict_proba(X_test_rf)[:, 1]
)
## calculate auc on testing set
print(f"AUC on test set is {functions.cal_auc(pentathlon_nptb_test, 'p_yes_rf')}")

AUC on test set is 0.8743


## Prediction on test set

In [124]:
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_endurance_rf=functions.cal_prb(message="endurance", model=rf, test_set=X_test_rf),
    p_strength_rf=functions.cal_prb(message="strength", model=rf, test_set=X_test_rf),
    p_water_rf=functions.cal_prb(message="water", model=rf, test_set=X_test_rf),
    p_team_rf=functions.cal_prb(message="team", model=rf, test_set=X_test_rf),
    p_backcountry_rf=functions.cal_prb(
        message="backcountry", model=rf, test_set=X_test_rf
    ),
    p_winter_rf=functions.cal_prb(message="winter", model=rf, test_set=X_test_rf),
    p_racquet_rf=functions.cal_prb(message="racquet", model=rf, test_set=X_test_rf),
)

In [125]:
p_set_rf = [
    "p_endurance_rf",
    "p_strength_rf",
    "p_water_rf",
    "p_team_rf",
    "p_backcountry_rf",
    "p_winter_rf",
    "p_racquet_rf",
]

In [126]:
pentathlon_nptb_test["p_target_rf"] = pentathlon_nptb_test[p_set_rf].max(axis=1)

In [127]:
# # choose the message that has the highest probability
pentathlon_nptb_test["to_offer_rf"] = (
    pentathlon_nptb_test[p_set_rf]
    .idxmax(axis=1)
    .str.replace("p_endurance_rf", "endurance")
    .replace("p_strength_rf", "strength")
    .replace("p_water_rf", "water")
    .replace("p_team_rf", "team")
    .replace("p_backcountry_rf", "backcountry")
    .replace("p_winter_rf", "racquet")
    .replace("p_racquet_rf", "winter")
)

In [128]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_rf"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_rf,
backcountry,"3,101"
endurance,196
racquet,"2,834"
strength,"11,618"
team,"3,445"
water,"4,940"
winter,"3,866"


In [129]:
pentathlon_nptb_test[
    [
        "p_target_rf",
        "p_endurance_rf",
        "p_strength_rf",
        "p_water_rf",
        "p_team_rf",
        "p_backcountry_rf",
        "p_winter_rf",
        "p_racquet_rf",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, perc=True)

p_target_rf         10.68%
p_strength_rf       10.11%
p_water_rf           9.69%
p_winter_rf          9.59%
p_racquet_rf         9.56%
p_team_rf            9.55%
p_backcountry_rf     9.42%
p_endurance_rf        8.9%
dtype: object

## Profit on test set

In [130]:
ep_set_rf = [
    "ep_endurance_rf",
    "ep_strength_rf",
    "ep_water_rf",
    "ep_team_rf",
    "ep_backcountry_rf",
    "ep_winter_rf",
    "ep_racquet_rf",
]
for i in range(7):
    pentathlon_nptb_test[ep_set_rf[i]] = pentathlon_nptb_test[p_set_rf[i]] * avg_os[i]

In [131]:
pentathlon_nptb_test["to_offer_ep_rf"] = (
    pentathlon_nptb_test[ep_set_rf]
    .idxmax(axis=1)
    .str.replace("ep_endurance_rf", "endurance")
    .replace("ep_strength_rf", "strength")
    .replace("ep_water_rf", "water")
    .replace("ep_team_rf", "team")
    .replace("ep_backcountry_rf", "backcountry")
    .replace("ep_winter_rf", "racquet")
    .replace("ep_racquet_rf", "winter")
)

In [132]:
pentathlon_nptb_test["ep_target_rf"] = pentathlon_nptb_test[ep_set_rf].max(axis=1)

In [133]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_ep_rf"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep_rf,
backcountry,"13,889"
racquet,"4,104"
strength,"4,009"
team,"2,585"
water,"2,893"
winter,"2,520"


In [134]:
pentathlon_nptb_test[
    [
        "ep_target_rf",
        "ep_endurance_rf",
        "ep_strength_rf",
        "ep_water_rf",
        "ep_team_rf",
        "ep_backcountry_rf",
        "ep_winter_rf",
        "ep_racquet_rf",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target_rf         $2.47
ep_backcountry_rf    $2.29
ep_winter_rf         $2.27
ep_strength_rf       $2.26
ep_water_rf          $2.22
ep_racquet_rf        $2.21
ep_team_rf            $2.2
ep_endurance_rf      $1.93
dtype: object

In [135]:
profit_rf = pentathlon_nptb_test["ep_target_rf"].agg("mean") * 30000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_rf)}")

Expected profit from offer customization: $74,163.8


## Random Forest with GridSearch

In [34]:
# params_set = {
#     "n_estimators": range(100, 1001, 50),
#     "max_depth": range(1, 11),
#     "min_samples_split": range(50, 400),
#     "min_samples_leaf": range(1, 60),
# }
# other_params = {
#     "bootstrap": True,
#     "random_state": 1234,
#     "criterion": "entropy",
# }
# rf_gs = RandomForestClassifier(**other_params)

# optimized_rf = RandomizedSearchCV(
#     estimator=rf_gs,
#     param_distributions=params_set,
#     scoring="roc_auc",
#     cv=2,
#     verbose=3,
#     n_jobs=-1,
#     n_iter=20,
# ).fit(
#     X_train_rf,
#     y_train_rf,
#     sample_weight=pentathlon_nptb_train["cweight"],
# )
# optimized_rf.best_estimator_

GridSearch shows that best parameters are: criterion='entropy', max_depth=10, min_samples_leaf=9, min_samples_split=89, n_estimators=250, random_state=1234.We will use these parameter to build the model again.

In [35]:
rf_gs = RandomForestClassifier(
    bootstrap=True,
    criterion="entropy",
    max_depth=10,
    min_samples_leaf=9,
    min_samples_split=89,
    n_estimators=250,
    random_state=1234,
).fit(
    X_train_rf,
    y_train_rf,
    sample_weight=pentathlon_nptb_train["cweight"],
)

## AUC on test set

In [36]:
# make prediction by using current message option
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_yes_rf_gs=rf_gs.predict_proba(X_test_rf)[:, 1]
)
## calculate auc on testing set
print(f"AUC on test set is {functions.cal_auc(pentathlon_nptb_test, 'p_yes_rf_gs')}")

AUC on test set is 0.8815


## Prediction on test set

In [37]:
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_endurance_rf_gs=functions.cal_prb(
        message="endurance", model=rf_gs, test_set=X_test_rf
    ),
    p_strength_rf_gs=functions.cal_prb(
        message="strength", model=rf_gs, test_set=X_test_rf
    ),
    p_water_rf_gs=functions.cal_prb(message="water", model=rf_gs, test_set=X_test_rf),
    p_team_rf_gs=functions.cal_prb(message="team", model=rf_gs, test_set=X_test_rf),
    p_backcountry_rf_gs=functions.cal_prb(
        message="backcountry", model=rf_gs, test_set=X_test_rf
    ),
    p_winter_rf_gs=functions.cal_prb(message="winter", model=rf_gs, test_set=X_test_rf),
    p_racquet_rf_gs=functions.cal_prb(
        message="racquet", model=rf_gs, test_set=X_test_rf
    ),
)

In [38]:
p_set_rf_gs = [
    "p_endurance_rf_gs",
    "p_strength_rf_gs",
    "p_water_rf_gs",
    "p_team_rf_gs",
    "p_backcountry_rf_gs",
    "p_winter_rf_gs",
    "p_racquet_rf_gs",
]

In [39]:
pentathlon_nptb_test["p_target_rf_gs"] = pentathlon_nptb_test[p_set_rf_gs].max(axis=1)

In [40]:
# # choose the message that has the highest probability
pentathlon_nptb_test["to_offer_rf_gs"] = (
    pentathlon_nptb_test[p_set_rf_gs]
    .idxmax(axis=1)
    .str.replace("p_endurance_rf_gs", "endurance")
    .replace("p_strength_rf_gs", "strength")
    .replace("p_water_rf_gs", "water")
    .replace("p_team_rf_gs", "team")
    .replace("p_backcountry_rf_gs", "backcountry")
    .replace("p_winter_rf_gs", "racquet")
    .replace("p_racquet_rf_gs", "winter")
)

In [41]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_rf_gs"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_rf_gs,
backcountry,"1,902"
endurance,305
racquet,"2,202"
strength,"14,307"
team,"2,887"
water,"5,398"
winter,"2,999"


In [42]:
pentathlon_nptb_test[
    [
        "p_target_rf_gs",
        "p_endurance_rf_gs",
        "p_strength_rf_gs",
        "p_water_rf_gs",
        "p_team_rf_gs",
        "p_backcountry_rf_gs",
        "p_winter_rf_gs",
        "p_racquet_rf_gs",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, perc=True)

p_target_rf_gs         7.99%
p_strength_rf_gs       7.82%
p_water_rf_gs          7.48%
p_winter_rf_gs         7.45%
p_racquet_rf_gs        7.44%
p_backcountry_rf_gs    7.34%
p_team_rf_gs           7.31%
p_endurance_rf_gs       7.2%
dtype: object

## Profit on test set

In [43]:
ep_set_rf_gs = [
    "ep_endurance_rf_gs",
    "ep_strength_rf_gs",
    "ep_water_rf_gs",
    "ep_team_rf_gs",
    "ep_backcountry_rf_gs",
    "ep_winter_rf_gs",
    "ep_racquet_rf_gs",
]
for i in range(7):
    pentathlon_nptb_test[ep_set_rf_gs[i]] = (
        pentathlon_nptb_test[p_set_rf_gs[i]] * avg_os[i]
    )

In [44]:
pentathlon_nptb_test["to_offer_ep_rf_gs"] = (
    pentathlon_nptb_test[ep_set_rf_gs]
    .idxmax(axis=1)
    .str.replace("ep_endurance_rf_gs", "endurance")
    .replace("ep_strength_rf_gs", "strength")
    .replace("ep_water_rf_gs", "water")
    .replace("ep_team_rf_gs", "team")
    .replace("ep_backcountry_rf_gs", "backcountry")
    .replace("ep_winter_rf_gs", "racquet")
    .replace("ep_racquet_rf_gs", "winter")
)

In [45]:
pentathlon_nptb_test["ep_target_rf_gs"] = pentathlon_nptb_test[ep_set_rf_gs].max(axis=1)

In [46]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_ep_rf_gs"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep_rf_gs,
backcountry,"16,333"
racquet,"3,725"
strength,"4,421"
team,"1,630"
water,"2,484"
winter,"1,407"


In [47]:
pentathlon_nptb_test[
    [
        "ep_target_rf_gs",
        "ep_endurance_rf_gs",
        "ep_strength_rf_gs",
        "ep_water_rf_gs",
        "ep_team_rf_gs",
        "ep_backcountry_rf_gs",
        "ep_winter_rf_gs",
        "ep_racquet_rf_gs",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target_rf_gs         $1.85
ep_backcountry_rf_gs    $1.78
ep_winter_rf_gs         $1.76
ep_strength_rf_gs       $1.75
ep_racquet_rf_gs        $1.72
ep_water_rf_gs          $1.71
ep_team_rf_gs           $1.68
ep_endurance_rf_gs      $1.56
dtype: object

In [48]:
profit_rf_gs = pentathlon_nptb_test["ep_target_rf_gs"].agg("mean") * 30000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_rf_gs)}")

Expected profit from offer customization: $55,427.17


# Gradient Boosting

In [49]:
# columns used to build models
variables_gb = [
    "message",
    "age",
    "gender",
    "income_ln",
    "education",
    "children",
    "freq_endurance",
    "freq_strength",
    "freq_water",
    "freq_team",
    "freq_backcountry",
    "freq_winter",
    "freq_racquet",
]

# split data into training and testing data
X_train_gb = pentathlon_nptb_train[variables_gb]
y_train_gb = pentathlon_nptb_train["buyer_yes"]

X_test_gb = pentathlon_nptb_test[variables_gb]
y_test_gb = pentathlon_nptb_test["buyer_yes"]

# get dummy for predictors
X_train_gb = pd.get_dummies(X_train_gb, drop_first=True)
X_test_gb = pd.get_dummies(X_test_gb, drop_first=True)

In [50]:
gb = GradientBoostingClassifier(random_state=0).fit(
    X_train_gb, y_train_gb, sample_weight=pentathlon_nptb_train["cweight"]
)

## AUC on test set

In [51]:
# make prediction by using current message option
pentathlon_nptb_test["p_yes_gb"] = gb.predict_proba(X_test_gb)[:, 1]
## calculate auc on testing set
print(f"AUC on test set is {functions.cal_auc(pentathlon_nptb_test, 'p_yes_gb')}")

AUC on test set is 0.8791


## Prediction on test set

In [52]:
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_endurance_gb=functions.cal_prb(message="endurance", model=gb, test_set=X_test_gb),
    p_strength_gb=functions.cal_prb(message="strength", model=gb, test_set=X_test_gb),
    p_water_gb=functions.cal_prb(message="water", model=gb, test_set=X_test_gb),
    p_team_gb=functions.cal_prb(message="team", model=gb, test_set=X_test_gb),
    p_backcountry_gb=functions.cal_prb(
        message="backcountry", model=gb, test_set=X_test_gb
    ),
    p_winter_gb=functions.cal_prb(message="winter", model=gb, test_set=X_test_gb),
    p_racquet_gb=functions.cal_prb(message="racquet", model=gb, test_set=X_test_gb),
)

In [53]:
p_set_gb = [
    "p_endurance_gb",
    "p_strength_gb",
    "p_water_gb",
    "p_team_gb",
    "p_backcountry_gb",
    "p_racquet_gb",
    "p_winter_gb",
]
pentathlon_nptb_test["p_target_gb"] = pentathlon_nptb_test[p_set_gb].max(axis=1)

In [54]:
# choose message that has the highest probability
pentathlon_nptb_test["to_offer_gb"] = (
    pentathlon_nptb_test[p_set_gb]
    .idxmax(axis=1)
    .str.replace("p_endurance_gb", "endurance")
    .replace("p_strength_gb", "strength")
    .replace("p_water_gb", "water")
    .replace("p_team_gb", "team")
    .replace("p_backcountry_gb", "backcountry")
    .replace("p_racquet_gb", "racquet")
    .replace("p_winter_gb", "winter")
)

In [55]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_gb"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_gb,
backcountry,155
endurance,"24,558"
racquet,114
strength,"5,057"
water,2
winter,114


In [56]:
pentathlon_nptb_test[
    [
        "p_target_gb",
        "p_endurance_gb",
        "p_strength_gb",
        "p_water_gb",
        "p_team_gb",
        "p_backcountry_gb",
        "p_racquet_gb",
        "p_winter_gb",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, perc=True)

p_target_gb          6.2%
p_strength_gb       6.12%
p_backcountry_gb    5.54%
p_racquet_gb        5.49%
p_winter_gb         5.47%
p_endurance_gb      5.44%
p_water_gb          5.43%
p_team_gb           5.41%
dtype: object

## Profit on test set

In [57]:
ep_set_gb = [
    "ep_endurance_gb",
    "ep_strength_gb",
    "ep_water_gb",
    "ep_team_gb",
    "ep_backcountry_gb",
    "ep_racquet_gb",
    "ep_winter_gb",
]
for i in range(7):
    pentathlon_nptb_test[ep_set_gb[i]] = pentathlon_nptb_test[p_set_gb[i]] * avg_os[i]

In [58]:
pentathlon_nptb_test["to_offer_ep_gb"] = (
    pentathlon_nptb_test[ep_set_gb]
    .idxmax(axis=1)
    .str.replace("ep_endurance_gb", "endurance")
    .replace("ep_strength_gb", "strength")
    .replace("ep_water_gb", "water")
    .replace("ep_team_gb", "team")
    .replace("ep_backcountry_gb", "backcountry")
    .replace("ep_winter_gb", "racquet")
    .replace("ep_racquet_gb", "winter")
)
pentathlon_nptb_test["ep_target_gb"] = pentathlon_nptb_test[ep_set_gb].max(axis=1)

In [59]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_ep_gb"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep_gb,
backcountry,"25,470"
racquet,92
strength,"4,279"
water,2
winter,157


In [60]:
pentathlon_nptb_test[
    [
        "ep_target_gb",
        "ep_endurance_gb",
        "ep_strength_gb",
        "ep_water_gb",
        "ep_team_gb",
        "ep_backcountry_gb",
        "ep_racquet_gb",
        "ep_winter_gb",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target_gb         $1.43
ep_strength_gb       $1.37
ep_backcountry_gb    $1.35
ep_racquet_gb         $1.3
ep_winter_gb         $1.26
ep_team_gb           $1.25
ep_water_gb          $1.24
ep_endurance_gb      $1.18
dtype: object

In [61]:
profit_gb = pentathlon_nptb_test["ep_target_gb"].agg("mean") * 30000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_gb)}")

Expected profit from offer customization: $42,972.6


## Gradient Boosting with GridSeach

### Hyper tuning on learning rate, max_depth, n_estimators

since run codes of building gradient boosting with GridSearch takes long time, we have commented them.

In [62]:
# gb_tuning = GradientBoostingClassifier(random_state = 0)
# parameters = {
#     "n_estimators":[5,50,250,500],
#     "max_depth":[1,3,5,7,9],
#     "learning_rate":[0.01,0.1,1,10,100]
# }
# scoring = {"AUC": "roc_auc"}

# gb_cv = GridSearchCV(gb_tuning,parameters,scoring = scoring,
#                      cv=5, n_jobs = 4, refit = 'AUC', verbose = 5)
# gb_cv.fit(X_train,y_train)

In [63]:
# gb_cv = GridSearchCV(gb_tuning,parameters,scoring = scoring, cv=5, n_jobs = 4, refit = 'AUC', verbose = 5)
# gb_cv.fit(X_train,y_train)

In [64]:
# print(gb_cv.best_params_)
# print(gb_cv.best_score_)

In [65]:
# # results from gradient boosting CV on learning rate, max depth, and n_estimators
# gb_cv_results = pd.DataFrame(gb_cv.cv_results_).sort_values(by="rank_test_AUC")

# gb_cv_results.iloc[0, gb_cv_results.columns.get_loc("param_max_depth")]

GridSearch shows that best parameters are 0.01 for learning_rate, 7 for max_depth, 500 for n_estimators.We will use these parameter to build the model again. 

In [66]:
gb_gs = GradientBoostingClassifier(
    learning_rate=0.01, max_depth=7, n_estimators=500, random_state=0
).fit(X_train_gb, y_train_gb, sample_weight=pentathlon_nptb_train["cweight"])

## AUC on test set

In [67]:
# make prediction by using current message option
pentathlon_nptb_test["p_yes_gb_gs"] = gb_gs.predict_proba(X_test_gb)[:, 1]
## calculate auc on testing set
print(f"AUC on test set is {functions.cal_auc(pentathlon_nptb_test, 'p_yes_gb_gs')}")

AUC on test set is 0.8799


## Prediction on test set

In [68]:
pentathlon_nptb_test = pentathlon_nptb_test.assign(
    p_endurance_gb_gs=functions.cal_prb(
        message="endurance", model=gb_gs, test_set=X_test_gb
    ),
    p_strength_gb_gs=functions.cal_prb(
        message="strength", model=gb_gs, test_set=X_test_gb
    ),
    p_water_gb_gs=functions.cal_prb(message="water", model=gb_gs, test_set=X_test_gb),
    p_team_gb_gs=functions.cal_prb(message="team", model=gb_gs, test_set=X_test_gb),
    p_backcountry_gb_gs=functions.cal_prb(
        message="backcountry", model=gb_gs, test_set=X_test_gb
    ),
    p_winter_gb_gs=functions.cal_prb(message="winter", model=gb_gs, test_set=X_test_gb),
    p_racquet_gb_gs=functions.cal_prb(
        message="racquet", model=gb_gs, test_set=X_test_gb
    ),
)

In [69]:
p_set_gb_gs = [
    "p_endurance_gb_gs",
    "p_strength_gb_gs",
    "p_water_gb_gs",
    "p_team_gb_gs",
    "p_backcountry_gb_gs",
    "p_racquet_gb_gs",
    "p_winter_gb_gs",
]
pentathlon_nptb_test["p_target_gb_gs"] = pentathlon_nptb_test[p_set_gb].max(axis=1)

In [70]:
# # choose the message that has the highest probability and save it to a variable named 'to_offer_gb_gs'
pentathlon_nptb_test["to_offer_gb_gs"] = (
    pentathlon_nptb_test[p_set_gb_gs]
    .idxmax(axis=1)
    .str.replace("p_endurance_gb_gs", "endurance")
    .replace("p_strength_gb_gs", "strength")
    .replace("p_water_gb_gs", "water")
    .replace("p_team_gb_gs", "team")
    .replace("p_backcountry_gb_gs", "backcountry")
    .replace("p_racquet_gb_gs", "racquet")
    .replace("p_winter_gb_gs", "winter")
)

In [71]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_gb_gs"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_gb_gs,
backcountry,481
endurance,"14,319"
racquet,494
strength,"10,363"
team,"1,189"
water,"2,337"
winter,817


In [72]:
pentathlon_nptb_test[p_set_gb_gs].agg("mean").sort_values(ascending=False).apply(
    rsm.format_nr, perc=True
)

p_strength_gb_gs       9.95%
p_water_gb_gs          9.68%
p_team_gb_gs           9.65%
p_winter_gb_gs         9.52%
p_racquet_gb_gs        9.42%
p_backcountry_gb_gs    9.25%
p_endurance_gb_gs      9.13%
dtype: object

## Profit on test set

In [73]:
ep_set_gb_gs = [
    "ep_endurance_gb_gs",
    "ep_strength_gb_gs",
    "ep_water_gb_gs",
    "ep_team_gb_gs",
    "ep_backcountry_gb_gs",
    "ep_racquet_gb_gs",
    "ep_winter_gb_gs",
]
for i in range(7):
    pentathlon_nptb_test[ep_set_gb_gs[i]] = (
        pentathlon_nptb_test[p_set_gb_gs[i]] * avg_os[i]
    )

In [74]:
pentathlon_nptb_test["to_offer_ep_gb_gs"] = (
    pentathlon_nptb_test[ep_set_gb_gs]
    .idxmax(axis=1)
    .str.replace("p_endurance_gb_gs", "endurance")
    .replace("p_strength_gb_gs", "strength")
    .replace("p_water_gb_gs", "water")
    .replace("p_team_gb_gs", "team")
    .replace("p_backcountry_gb_gs", "backcountry")
    .replace("p_racquet_gb_gs", "racquet")
    .replace("p_winter_gb_gs", "winter")
)
pentathlon_nptb_test["ep_target_gb_gs"] = pentathlon_nptb_test[ep_set_gb_gs].max(axis=1)

In [75]:
pd.crosstab(index=pentathlon_nptb_test["to_offer_ep_gb_gs"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep_gb_gs,
ep_backcountry_gb_gs,"24,172"
ep_racquet_gb_gs,939
ep_strength_gb_gs,"2,035"
ep_team_gb_gs,"1,060"
ep_water_gb_gs,"1,127"
ep_winter_gb_gs,667


In [76]:
pentathlon_nptb_test[
    [
        "ep_target_gb_gs",
        "ep_endurance_gb_gs",
        "ep_strength_gb_gs",
        "ep_water_gb_gs",
        "ep_team_gb_gs",
        "ep_backcountry_gb_gs",
        "ep_racquet_gb_gs",
        "ep_winter_gb_gs",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target_gb_gs         $2.59
ep_backcountry_gb_gs    $2.24
ep_racquet_gb_gs        $2.23
ep_strength_gb_gs       $2.23
ep_team_gb_gs           $2.22
ep_water_gb_gs          $2.22
ep_winter_gb_gs          $2.2
ep_endurance_gb_gs      $1.98
dtype: object

In [77]:
profit_gb_gs = pentathlon_nptb_test["ep_target_gb_gs"].agg("mean") * 30000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_gb_gs)}")

Expected profit from offer customization: $77,566.26


In [136]:
profit_df = pd.DataFrame(
    {
        "Name": [
            "LR",
            "RF",
            "RF_GS",
            "GB",
            "GB_GS",
        ],
        "AUC": [
            functions.cal_auc(pentathlon_nptb_test, "p_yes_lr"),
            functions.cal_auc(pentathlon_nptb_test, "p_yes_rf"),
            functions.cal_auc(pentathlon_nptb_test, "p_yes_rf_gs"),
            functions.cal_auc(pentathlon_nptb_test, "p_yes_gb"),
            functions.cal_auc(pentathlon_nptb_test, "p_yes_gb_gs"),
        ],
    }
)
profit_df

,Name,AUC
0,LR,0.8760
1,RF,0.8743
2,RF_GS,0.8815
3,GB,0.8791
4,GB_GS,0.8799


# Profit on representative set

In [83]:
pentathlon_nptb_rep = pentathlon_nptb_copy.query("representative == 1")

In [84]:
model_select = rf_gs

## Predicted probability

In [85]:
X = pd.get_dummies(pentathlon_nptb_rep[variables_gb], drop_first=True)

In [86]:
pentathlon_nptb_rep = pentathlon_nptb_rep.assign(
    p_endurance=functions.cal_prb(message="endurance", model=model_select, test_set=X),
    p_strength=functions.cal_prb(message="strength", model=model_select, test_set=X),
    p_water=functions.cal_prb(message="water", model=model_select, test_set=X),
    p_team=functions.cal_prb(message="team", model=model_select, test_set=X),
    p_backcountry=functions.cal_prb(
        message="backcountry", model=model_select, test_set=X
    ),
    p_winter=functions.cal_prb(message="winter", model=model_select, test_set=X),
    p_racquet=functions.cal_prb(message="racquet", model=model_select, test_set=X),
)

In [87]:
p_set = [
    "p_endurance",
    "p_strength",
    "p_water",
    "p_team",
    "p_backcountry",
    "p_winter",
    "p_racquet",
]
pentathlon_nptb_rep["p_target"] = pentathlon_nptb_rep[p_set].max(axis=1)

In [88]:
# # choose the message that has the highest probability
pentathlon_nptb_rep["to_offer"] = (
    pentathlon_nptb_rep[p_set]
    .idxmax(axis=1)
    .str.replace("p_endurance", "endurance")
    .replace("p_strength", "strength")
    .replace("p_water", "water")
    .replace("p_team", "team")
    .replace("p_backcountry", "backcountry")
    .replace("p_winter", "racquet")
    .replace("p_racquet", "winter")
)

In [89]:
pd.crosstab(index=pentathlon_nptb_rep["to_offer"], columns="count").apply(rsm.format_nr)

col_0,count
to_offer,
backcountry,"6,528"
endurance,"1,969"
racquet,"4,850"
strength,"48,977"
team,"7,618"
water,"18,833"
winter,"11,225"


In [90]:
pentathlon_nptb_rep["p_target"] = pentathlon_nptb_rep[p_set].max(axis=1)
pentathlon_nptb_rep[
    [
        "p_endurance",
        "p_strength",
        "p_water",
        "p_team",
        "p_backcountry",
        "p_winter",
        "p_racquet",
        "p_target",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, perc=True)

p_target         1.72%
p_strength       1.66%
p_water          1.61%
p_winter         1.58%
p_racquet        1.58%
p_team           1.57%
p_backcountry    1.56%
p_endurance      1.53%
dtype: object

## Expected profit

In [91]:
ep_set = [
    "ep_endurance",
    "ep_strength",
    "ep_water",
    "ep_team",
    "ep_backcountry",
    "ep_winter",
    "ep_racquet",
]

In [92]:
for i in range(7):
    pentathlon_nptb_rep[ep_set[i]] = pentathlon_nptb_rep[p_set[i]] * avg_os[i]
pentathlon_nptb_rep["to_offer_ep"] = (
    pentathlon_nptb_rep[ep_set]
    .idxmax(axis=1)
    .str.replace("ep_endurance", "endurance")
    .replace("ep_strength", "strength")
    .replace("ep_water", "water")
    .replace("ep_team", "team")
    .replace("ep_backcountry", "backcountry")
    .replace("ep_winter", "racquet")
    .replace("ep_racquet", "winter")
)
pentathlon_nptb_rep["ep_target"] = pentathlon_nptb_rep[ep_set].max(axis=1)

In [93]:
pd.crosstab(index=pentathlon_nptb_rep["to_offer_ep"], columns="count").apply(
    rsm.format_nr
)

col_0,count
to_offer_ep,
backcountry,"69,785"
racquet,"7,996"
strength,"9,786"
team,"3,298"
water,"6,159"
winter,"2,976"


In [94]:
pentathlon_nptb_rep[
    [
        "ep_target",
        "ep_endurance",
        "ep_strength",
        "ep_water",
        "ep_team",
        "ep_backcountry",
        "ep_racquet",
        "ep_winter",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_target          $0.4
ep_backcountry    $0.38
ep_winter         $0.37
ep_strength       $0.37
ep_water          $0.37
ep_racquet        $0.36
ep_team           $0.36
ep_endurance      $0.33
dtype: object

In [95]:
profit_rep = pentathlon_nptb_rep["ep_target"].agg("mean") * pentathlon_nptb_rep.shape[0]
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_rep)}")

Expected profit from offer customization: $39,689.94


In [96]:
profit_5M = pentathlon_nptb_rep["ep_target"].agg("mean") * 5000000
print(f"Expected profit from offer customization: ${rsm.format_nr(profit_5M)}")

Expected profit from offer customization: $1,984,497.15


## The Analysis

1. For each customer determine the message (i.e., endurance, strength, water, team, backcountry, winter, or racquet) predicted to lead to the highest probability of purchase. Describe your approach.

In [97]:
pd.DataFrame(pentathlon_nptb_rep[["custid", "to_offer"]])

,custid,to_offer
0,U45198803,strength
1,U22197752,strength
3,U19423462,backcountry
4,U23888305,strength
6,U16954857,strength
...,...,...
199991,U12620333,endurance
199993,U18623424,strength
199994,U64468968,team
199998,U33721691,strength


'to_offer' columns has the message type that has the highest probability

2. For each message, report the percentage of customers for whom that message maximizes their probability of purchase.

In [98]:
pd.crosstab(
    index=pentathlon_nptb_rep["to_offer"],
    columns="Percentage of customers",
    normalize=True,
    colnames=[""],
).apply(rsm.format_nr, perc=True)

,Percentage of customers
to_offer,
backcountry,6.53%
endurance,1.97%
racquet,4.85%
strength,48.98%
team,7.62%
water,18.83%
winter,11.22%


The table above shows that strength message type has the highest percentage of customers that message maximizes their purchase probability while endurance message type has the lowest percentage of customers. 

3. For each customer, determine the message (i.e., endurance, strength, water, team, backcountry, winter, or racquet) predicted to lead to the highest expected profit (COGS is 60%). Describe your approach to predict order size and how you calculated expected profit.

In [99]:
pentathlon_nptb_rep[["custid", "to_offer_ep"]]

,custid,to_offer_ep
0,U45198803,backcountry
1,U22197752,backcountry
3,U19423462,backcountry
4,U23888305,strength
6,U16954857,backcountry
...,...,...
199991,U12620333,backcountry
199993,U18623424,backcountry
199994,U64468968,backcountry
199998,U33721691,backcountry


We predict average order size by calculate average total_os by grouping each message type on training set. We calculate expected profit by multiplying probability, 0.4 (1-COGS, 0.6) and average order size. 

4. Report for each message, i.e., endurance, racket, etc., the percentage of customers for whom that message maximizes their expected profit.

In [100]:
pd.crosstab(
    index=pentathlon_nptb_rep["to_offer_ep"],
    columns="Percentage of customers",
    colnames=[""],
    normalize=True,
).apply(rsm.format_nr, perc=True)

,Percentage of customers
to_offer_ep,
backcountry,69.78%
racquet,8.0%
strength,9.79%
team,3.3%
water,6.16%
winter,2.98%


The table above shows that backcountry message type has the highest percentage of customers that message maximizes their expected profit while endurance message type has the lowest percentage of customers, 0%. 

5. What expected profit can we obtain, on average, per e-mailed customer if we customize the message to each customer?

In [101]:
pentathlon_nptb_rep[["ep_target",]].agg("mean").sort_values(
    ascending=False
).apply(rsm.format_nr, sym="$")

ep_target    $0.4
dtype: object

If we customize the message to each cutomer, the average expected profit per e-mailed customer is $0.4

6. What is the expected profit per e-mailed customer if every customer receives the same message? Answer this question for each of the seven possible messages (i.e., endurance, strength, water, team, backcountry, winter, or racquet).

In [102]:
pentathlon_nptb_rep[
    [
        "ep_endurance",
        "ep_strength",
        "ep_water",
        "ep_team",
        "ep_backcountry",
        "ep_winter",
        "ep_racquet",
    ]
].agg("mean").sort_values(ascending=False).apply(rsm.format_nr, sym="$")

ep_backcountry    $0.38
ep_winter         $0.37
ep_strength       $0.37
ep_water          $0.37
ep_racquet        $0.36
ep_team           $0.36
ep_endurance      $0.33
dtype: object

As the chart above shows, the average expected profit per e-emailed customer if customers receive endurance message type is the lowest, \\$0.33 while the average expected profit per emailed customer if customers receive backcountry message type is the highest, \\$0.38

7. What is the expected profit per e-mailed customer if every customer is assigned randomly to one of the seven messages?

In [103]:
pentathlon_nptb_rep["total_os"].mean() * 0.4

0.2213697200000001

The expected profit per e-mailed customer is $0.2214 if customers are assigned randomly.

8. For the typical promotional e-mail blast to 5,000,000 customers, what improvement (in percent and in total Euros) could Pentathlon achieve by customizing the message to each customer rather than assigning customers a message randomly?

In [104]:
customized_profit_5M = pentathlon_nptb_rep["ep_target"].agg("mean") * 5000000
random_profit_5M = pentathlon_nptb_rep["total_os"].mean() * 0.4 * 5000000
print(customized_profit_5M - random_profit_5M)

877648.552511106


In [105]:
percent_increase = (customized_profit_5M - random_profit_5M) / random_profit_5M
percent_increase

0.7929255658914015

The increase of customizing the message to each customer will be $877648.55 or 79.29%

9. Comment on the new e-mail policy proposal. What are its weaknesses? Suggest at least one improvement?

The weakness of this policy lies in the imbalance of profitability between different departments. Suppose we have two message type that generates a higher expected profit this period, we will only send these two to a customer in the next period. We can imagine that these two will still have higher chances to gain more profits with more exposure to customers. In this case, the Matthew Effect occurs, which means the proportion of certain types of message will increase continuously and the lower-profit message types will disappear eventually.  Besides, if we keep sending the same type of message to a customer, they might find them tedious and ignore them.  

Improvement: To avoid this Matthew Effect, we can adopt a new strategy, which means each time we sent two messages to our customers: one with the high expected profit, and the other with a low expected profit. Or we can re-write messages for customers, in each message, we recommend a product with high expected profit and a product with low expected profit. By doing this, products with low expected profit have more chance to be seen by customers.